In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Define the stateful custom Dense layer
class MyDenseLayer(layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[-1], self.units),
                                      initializer='glorot_uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias',
                                    shape=(self.units,),
                                    initializer='zeros',
                                    trainable=True)
        super().build(input_shape)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': keras.activations.serialize(self.activation),
        })
        return config

# Generate random data
x_train = np.random.rand(1000, 20)
y_train = np.random.randint(2, size=(1000, 1))

x_test = np.random.rand(200, 20)
y_test = np.random.randint(2, size=(200, 1))

# Define and compile the model
model = keras.Sequential([
    MyDenseLayer(64, activation='relu', input_shape=(20,)),
    MyDenseLayer(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/5


C:\Users\Harshith\AppData\Local\Temp\ipykernel_4124\306574571.py:9: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5150 - loss: 0.6973 - val_accuracy: 0.5050 - val_loss: 0.6970
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4988 - loss: 0.6961 - val_accuracy: 0.4850 - val_loss: 0.6968
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5660 - loss: 0.6864 - val_accuracy: 0.4700 - val_loss: 0.6981
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5531 - loss: 0.6836 - val_accuracy: 0.4850 - val_loss: 0.6996
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5552 - loss: 0.6841 - val_accuracy: 0.4700 - val_loss: 0.7010
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5289 - loss: 0.6953 
Test accuracy: 0.5350000262260437


In [7]:
import tensorflow as tf
from tensorflow import keras

class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(tf.shape(inputs), stddev=self.stddev)
            return inputs + noise
        else:
            return inputs

    def compute_output_shape(self, input_shape):
        return input_shape

# Create an instance of the model
model = keras.Sequential([
    MyGaussianNoise(stddev=0.1)
])

# Test the model with random inputs
input_data = tf.random.normal((1, 10))  # Example input data

# Test with training=True (adds noise)
output_with_noise = model(input_data, training=True)
print("Output with Noise (training=True):")
print(output_with_noise)

# Test with training=False (no noise added)
output_without_noise = model(input_data, training=False)
print("\nOutput without Noise (training=False):")
print(output_without_noise)

# Print summary of the model
model.summary()


Output with Noise (training=True):
tf.Tensor(
[[-1.0244232  -0.35358804  0.7700415  -1.0950521   0.18652976  0.34801552
   0.3602058   0.20025687  0.4768229  -0.83598006]], shape=(1, 10), dtype=float32)

Output without Noise (training=False):
tf.Tensor(
[[-0.9965972  -0.2909648   0.7868603  -1.1508238   0.1823237   0.4465502
   0.43137014 -0.00764798  0.70394623 -0.7575801 ]], shape=(1, 10), dtype=float32)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_gaussian_noise_1             │ (1, 10)                │             0 │
│ (MyGaussianNoise)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [1]:
import tensorflow as tf
from tensorflow import keras

# Define the ResidualBlock layer
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

# Define the ResidualRegressor model
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
            Z = self.block2(Z)
        return self.out(Z)

# Create an instance of the ResidualRegressor model
model = ResidualRegressor(output_dim=1)

# Compile the model (optional, if you plan to train it)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Display model summary
model.build((None, 10))  # Specify input shape (None for batch size, 10 for input size)
model.summary()


C:\Users\Harshith\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:372: UserWarning: `build()` was called on layer 'residual_regressor', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "residual_regressor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block (ResidualBlock)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ residual_block_1                │ ?                      │   0 (unbuilt) │
│ (ResidualBlock)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)